In [ ]:
from rdflib import Graph

# create an empty graph and parse the TTL file
g = Graph()
g.parse('./ontology/ma-ontology.ttl', format='turtle')

print(f"Loaded {len(g)} triples from ontology.")


Loaded 196 triples from ontology.


In [ ]:
import json

with open('./data/sample_ma_planning.json') as f:
    data = json.load(f)

print("Keys in JSON:", list(data.keys()))


Keys in JSON: ['agents', 'agentStates', 'collisionEvents', 'conflictAlerts', 'agentSubPlans', 'jointPlans']


In [3]:
from rdflib import Namespace, URIRef, Literal, RDF, XSD

MA   = Namespace('http://example.org/ma-planning#')
PLAN = Namespace('https://ai4society.github.io/planning-ontology#')

# Bind prefixes for readability in serializations and queries
g.bind('ma',   MA)
g.bind('plan', PLAN)
g.bind('rdf',  RDF)
g.bind('xsd',  XSD)


In [4]:
# 5.1 Agents
for a in data['agents']:
    uri = URIRef(a['uri'])
    g.add((uri, RDF.type, MA.Agent))
    g.add((uri, MA.hasAgentID, Literal(a['agentID'])))
    g.add((uri, MA.hasCapability, Literal(a['capability'])))
    g.add((uri, MA.hasInitialLocation, URIRef(a['initialLocation'])))
    g.add((uri, MA.hasGoalLocation, URIRef(a['goalLocation'])))

# 5.2 AgentStates
for s in data['agentStates']:
    uri = URIRef(s['uri'])
    g.add((uri, RDF.type, MA.AgentState))
    g.add((uri, MA.agentAt, URIRef(s['agentAt'])))
    g.add((uri, MA.belongsToAgent, URIRef(s['agent'])))

# 5.3 CollisionEvents
for e in data['collisionEvents']:
    uri = URIRef(e['uri'])
    g.add((uri, RDF.type, MA.CollisionEvent))
    g.add((uri, MA.conflictID, Literal(e['conflictID'])))
    g.add((uri, MA.occursAtTime, Literal(e['occursAtTime'], datatype=XSD.dateTime)))
    g.add((uri, MA.conflictTypeEvent, Literal(e['conflictType'])))
    g.add((uri, MA.conflictLocation, URIRef(e['conflictLocation'])))
    for ag in e['involvesAgents']:
        g.add((uri, MA.involvesAgentsEvent, URIRef(ag)))

# 5.4 ConflictAlerts
for c in data['conflictAlerts']:
    uri = URIRef(c['uri'])
    g.add((uri, RDF.type, MA.ConflictAlert))
    g.add((uri, MA.alertsConflict, URIRef(c['alertsConflict'])))
    g.add((uri, MA.targetAgent, URIRef(c['targetAgent'])))
    g.add((uri, MA.rewindWindow, Literal(c['rewindWindow'], datatype=XSD.integer)))
    g.add((uri, MA.alertType, Literal(c['alertType'])))
    for cell in c.get('staticForbiddenCells', []):
        g.add((uri, MA.staticForbiddenCells, URIRef(cell)))

# 5.5 AgentSubPlans
for sp in data['agentSubPlans']:
    uri = URIRef(sp['uri'])
    g.add((uri, RDF.type, MA.AgentSubPlan))
    g.add((uri, MA.belongsToAgent, URIRef(sp['belongsToAgent'])))
    g.add((uri, MA.hasPlanCost, Literal(sp['planCost'], datatype=XSD.decimal)))
    if 'isDerivedFromConflict' in sp:
        g.add((uri, MA.isDerivedFromConflict, URIRef(sp['isDerivedFromConflict'])))
    g.add((uri, MA.planData, Literal(sp['planData'])))

# 5.6 JointPlans
for jp in data['jointPlans']:
    uri = URIRef(jp['uri'])
    g.add((uri, RDF.type, MA.JointPlan))
    for sub in jp['composedOfSubPlans']:
        g.add((uri, MA.composedOfSubPlans, URIRef(sub)))
    g.add((uri, MA.hasGlobalMakespan, Literal(jp['globalMakespan'], datatype=XSD.decimal)))
    for al in jp.get('usedAlertSet', []):
        g.add((uri, MA.usedAlertSet, URIRef(al)))
    g.add((uri, MA.planData, Literal(jp['planData'])))

print(f"Combined graph now has {len(g)} triples.")


Combined graph now has 240 triples.


In [5]:
queries = {
    "List all Agents": """
PREFIX ma:   <http://example.org/ma-planning#>
SELECT ?agent ?agentID ?capability ?initialLoc ?goalLoc WHERE {
  ?agent a ma:Agent ;
         ma:hasAgentID        ?agentID ;
         ma:hasCapability     ?capability ;
         ma:hasInitialLocation ?initialLoc ;
         ma:hasGoalLocation   ?goalLoc .
}
""",
    "AgentSubPlans for Agent1": """
PREFIX ma: <http://example.org/ma-planning#>
SELECT ?subplan ?cost ?conflict WHERE {
  ?subplan a ma:AgentSubPlan ;
           ma:belongsToAgent ma:Agent1 ;
           ma:hasPlanCost    ?cost .
  OPTIONAL { ?subplan ma:isDerivedFromConflict ?conflict }
}
""",
    "JointPlans & Makespan": """
PREFIX ma: <http://example.org/ma-planning#>
SELECT ?jp ?makespan (GROUP_CONCAT(?sub;separator=",") AS ?subs) WHERE {
  ?jp a ma:JointPlan ;
      ma:hasGlobalMakespan ?makespan ;
      ma:composedOfSubPlans ?sub .
}
GROUP BY ?jp ?makespan
""",
    "CollisionEvents": """
PREFIX ma: <http://example.org/ma-planning#>
SELECT ?event ?time ?type ?location (GROUP_CONCAT(?agent;separator=",") AS ?agents) WHERE {
  ?event a ma:CollisionEvent ;
         ma:occursAtTime        ?time ;
         ma:conflictTypeEvent   ?type ;
         ma:conflictLocation    ?location ;
         ma:involvesAgentsEvent ?agent .
}
GROUP BY ?event ?time ?type ?location
""",
    "ConflictAlerts": """
PREFIX ma: <http://example.org/ma-planning#>
SELECT ?alert ?agent ?event ?rewindWindow ?alertType WHERE {
  ?alert a ma:ConflictAlert ;
         ma:targetAgent    ?agent ;
         ma:alertsConflict ?event ;
         ma:rewindWindow   ?rewindWindow ;
         ma:alertType      ?alertType .
}
"""
}


In [11]:
# Neatly Display SPARQL Query Results without `ace_tools`

import pandas as pd
from IPython.display import display, Markdown

# Mapping full URIs to prefixes for readability
prefix_map = {
    "http://example.org/ma-planning#": "ma:",
    "https://ai4society.github.io/planning-ontology#": "plan:"
}

def simplify_term(term):
    """Replace known URI prefixes with short forms, else return str(term)."""
    s = str(term)
    for uri, prefix in prefix_map.items():
        if s.startswith(uri):
            return prefix + s[len(uri):]
    return s

# Sample `queries` dict (ensure this is defined as in previous cell)
# and `g` is your rdflib Graph containing the ontology + data.

for desc, q in queries.items():
    # Display query title
    display(Markdown(f"### {desc}"))
    
    # Execute query
    result = g.query(q)
    
    # Prepare DataFrame
    columns = [str(var) for var in result.vars]
    rows = [
        [simplify_term(val) for val in row]
        for row in result
    ]
    df = pd.DataFrame(rows, columns=columns)
    
    # Display table
    display(df)


### List all Agents

,agent,agentID,capability,initialLoc,goalLoc
0,ma:Agent1,A1,move,http://example.org/plan#Cell_0_0,http://example.org/plan#Cell_4_4
1,ma:Agent2,A2,move,http://example.org/plan#Cell_4_0,http://example.org/plan#Cell_0_4


### AgentSubPlans for Agent1

,subplan,cost,conflict
0,ma:SubPlan1,5,ma:Event1


### JointPlans & Makespan

,jp,makespan,subs
0,ma:JointPlan1,10,"ma:SubPlan1,http://example.org/ma-planning#Sub..."


### CollisionEvents

,event,time,type,location,agents
0,ma:Event1,2025-06-24T07:00:00,vertexConflict,http://example.org/plan#Cell_1_1,"ma:Agent1,http://example.org/ma-planning#Agent2"


### ConflictAlerts

,alert,agent,event,rewindWindow,alertType
0,ma:Alert1,ma:Agent1,ma:Event1,2,static
